# Pig to PySpark
* **Tools:** Ollama, Langchain, Autogen
* **TODO:**
  * Build user interface where PIG code can be copy/pasted or uploaded. 
  * User Autogen for creating CSV input data
  * Re-write as Python files (.py)

---
* **Next Steps:**
  * (05-08-2024) Looks like a popular opinion is ["LangGraph is better than AutoGen"](https://www.reddit.com/r/LangChain/comments/1b7q44y/autogen_vs_langgraph/?onetap_auto=true&one_tap=true). --> Swap out Autogen with LangGraph. 

---

## 0. Initial Setup 
1. generate Langchain API key.
2. TODO: How to safely save and load API keys
3.  https://docs.smith.langchain.com/

In [1]:
# tracing progress

import os 

os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = ''

In [2]:
# configure 
run_local = "Yes"
# local_llm = "mistral" # mistral: https://github.com/langchain-ai/langgraph/blob/main/examples/rag/langgraph_crag_local.ipynb
# local_llm = "mixtral"  # mixtral: https://scalastic.io/en/mixtral-ollama-llamaindex-llm/
local_llm = "llama3" # llama3: https://python.langchain.com/docs/integrations/chat/ollama/


In [3]:
import os

# Get the list of all files and directories in the current working directory
files_and_directories = os.listdir()

print(files_and_directories)

['docker-compose.yml', 'Dockerfile', 'file_handler.py', 'faq_generator.ipynb', 'setup-env.sh', 'langgraph_practice.ipynb', 'pig2pyspark_generator.ipynb', '.Trash-0', 'requirements.txt', '.env', 'coding', 'scripts', '.ipynb_checkpoints', '.gitignore', 'output', 'data', '.cache', '.git']


---

## RAG (Index?) - Uplaod Supporting Documents 
Not really needed for this project but as a placeholder add vector DB

In [4]:
from langchain_community.document_loaders import WebBaseLoader # this is for pulling 
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain_mistralai import MistralAIEmbeddings

# Load
url = "https://github.com/palantir/pyspark-style-guide"
loader = WebBaseLoader(url)
docs = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, chunk_overlap=100
)
all_splits = text_splitter.split_documents(docs)

# Embed and index
if run_local == "Yes":
    embedding = GPT4AllEmbeddings()
else:
    # embedding = MistralAIEmbeddings(mistral_api_key=mistral_api_key)
    pass

# Index
vectorstore = Chroma.from_documents(
    documents=all_splits,
    collection_name="rag-chroma",
    embedding=embedding,
)
retriever = vectorstore.as_retriever()

Downloading: 100%|██████████| 45.9M/45.9M [00:01<00:00, 39.8MiB/s]
Verifying: 100%|██████████| 45.9M/45.9M [00:00<00:00, 742MiB/s]


---

## LLMs 

We build two LLMs: 
1. PIG code --> benchmark input data 
2. PIG code --> PySpark code
   
* ref: https://python.langchain.com/docs/integrations/chat/ollama/
* JsonOutputParser: https://api.python.langchain.com/en/latest/output_parsers/langchain_core.output_parsers.json.JsonOutputParser.html
* OutputParser: https://medium.com/@larry_nguyen/langchain-101-lesson-3-output-parser-406591b094d7

In [5]:
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOllama
# from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.output_parsers import JsonOutputParser

# we use locally hosted llm models 
llm = ChatOllama(model='llama3', format="json", temperature=0)


## Create two templates: 
# 1. pig code to benchmark input data
prompt_data_gen = PromptTemplate(
    template="""
    You are an expert data scientist fluent in PIG and Python coding languages.
    Generate Python code that do the following: 
    1. Generate 20 lines or more CSV data that can be used to test the PIG code. 
       Ensure column names are consistent with the names in PIG code. 
    2. Write Python code that save this CSV data to the directory provided. 
        
    Here is the PIG code: \n\n {pig_code} \n\n
    Here is the directory to save CSV file: \n\n {sample_input_path} \n\n

    Give a string of Python code with correct indentation that can be ran to create and save CSV file to correct path. 
    Provide this as a JSON with a single key 'data_gen_code' and no premable or explaination.""",
    input_variables=["pig_code", "sample_input_path"],
)
sample_input_code_generator = prompt_data_gen | llm | JsonOutputParser()

prompt_data_regen = PromptTemplate(
    template="""
    You are an expert data scientist fluent in PIG and Python coding languages.
    Generate Python code that do the following: 
    * Debug and share updated Python code to generate 20 lines or more CSV data that can be used to thest the PIG code. 
    * Use the error message and the data that resulted in error as a reference to fix the Python code. 
        
    Here is the PIG code: \n\n {pig_code} \n\n
    Here is the Python code with error: \n\n {pycode_error} \n\n
    Here is the Python code error message: \n\n {pycode_error_message} \n\n
    Here is the directory to save CSV file: \n\n {sample_input_path} \n\n

    Give a string of Python code with correct indentation that can be ran to create and save CSV file to correct path. 
    Provide this as a JSON with a single key 'data_gen_code' and no premable or explaination.""",
    input_variables=["pig_code", "pycode_error", "pycode_error_message", "sample_input_path"],
)
fix_sample_input_code_generator = prompt_data_regen | llm | JsonOutputParser()


# 2. pig code to pyspark code 
prompt_pig2pyspark = PromptTemplate(
    template="""
    You are an expert data scientist fluent in PIG and PySpark coding languages.
    Generate PySpark code that do the following: 
    * Implement same logic and methods as the provided PIG code. 
    * When ran against a sample input data, outputs identical result as PIG code. 
        
    Here is the PIG code: \n\n {pig_code} \n\n

    Give a string of PySpark code with correct indentation. 
    Provide this as a JSON with a single key 'pyspark_code' and no premable or explaination.""",
    input_variables=["pig_code"],
)
pig_to_pyspark_converter = prompt_pig2pyspark | llm | JsonOutputParser()

prompt_pig2pyspark_regen = PromptTemplate(
    template="""
    You are an expert data scientist fluent in PIG and PySpark coding languages.
    Generate PySpark code that do the following: 
    * Implement same logic and methods as the provided PIG code. 
    * Use the PySpark code that returned an error message to update the PySpark code. 
    * Use the PySpark code error message to update the PySpark code. 
    * When ran against a sample input data, outputs identical result as PIG code. 
        
    Here is the PIG code: \n\n {pig_code} \n\n
    Here is the PySpark code with error: \n\n {pycode_error} \n\n
    Here is the PySpark code error message: \n\n {pycode_error_message} \n\n

    Give a string of PySpark code with correct indentation. 
    Provide this as a JSON with a single key 'pyspark_code' and no premable or explaination.""",
    input_variables=["pig_code", "pycode_error", "pycode_error_message"],
)
fix_pig_to_pyspark_converter = prompt_pig2pyspark | llm | JsonOutputParser()


In [6]:
## test with sample PIG code 
# load PIG code
pig_script_dir = './scripts/pig1.pig'

with open(pig_script_dir, 'r') as file:
    sample_pig_code = file.read()

print(sample_pig_code)
data_output_dir = './data'

########################################
datagen_code = sample_input_code_generator.invoke({"pig_code": sample_pig_code, "sample_input_path": data_output_dir})
print('*'*88)
print(datagen_code['data_gen_code'])
print('*'*88)

-- Load the data from a CSV file
transactions = LOAD 'data/sample1.csv' USING PigStorage(',') 
    AS (depStore:chararray, date:chararray, amount:int);

-- Filter transactions to include only those where the amount is greater than 200
high_value_transactions = FILTER transactions BY amount > 200;

-- Group the transactions by store
grouped_by_store = GROUP high_value_transactions BY depStore;

-- Calculate total and average sales per depStore
sales_summary = FOREACH grouped_by_store GENERATE 
    group AS depStore,
    SUM(high_value_transactions.amount) AS total_sales,
    AVG(high_value_transactions.amount) AS average_sales;

-- Store the summary in a CSV file
STORE sales_summary INTO 'output/sales_summary' USING PigStorage(',');

-- Optional: Just for demonstration, store filtered data to another directory
STORE high_value_transactions INTO 'output/high_value_transactions' USING PigStorage(',');



--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/callbacks/manager.py", line 1998, in _configure
    handler = LangChainTracer(
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 91, in __init__
    self.client = client or get_client()
  File "/usr/local/lib/python3.10/dist-packages/langchain_core/tracers/langchain.py", line 54, in get_client
    _CLIENT = Client()
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 534, in __init__
    _validate_api_key_if_hosted(self.api_url, self.api_key)
  File "/usr/local/lib/python3.10/dist-packages/langsmith/client.py", line 323, in _validate_api_key_if_hosted
    raise ls_utils.LangSmithUserError(
langsmith.utils.LangSmithUserError: API key must be provided when using hosted LangSmith API

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/pyth

ConnectionError: HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/chat (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7efff14cbac0>: Failed to establish a new connection: [Errno 111] Connection refused'))

**NOTE:** Run `$ollama pull model-name` before using. 

---

## AutoGen - Test Autogen for iterative code refinement 

In [9]:
from autogen import AssistantAgent, UserProxyAgent

In [16]:
config_list = [
  {
    "model": "llama3",
    "base_url": "http://localhost:11434/v1",
    "api_key": "",
  }
]

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False})
user_proxy.initiate_chat(assistant, message=f"""
Convert the following PIG code to PySpark code:

Apache Pig code: {sample_pig_code}

""")

user_proxy (to assistant):


Convert the following PIG code to PySpark code:

Apache Pig code: -- Load the data from a CSV file
transactions = LOAD 'data/sample1.csv' USING PigStorage(',') 
    AS (depStore:chararray, date:chararray, amount:int);

-- Filter transactions to include only those where the amount is greater than 200
high_value_transactions = FILTER transactions BY amount > 200;

-- Group the transactions by store
grouped_by_store = GROUP high_value_transactions BY depStore;

-- Calculate total and average sales per depStore
sales_summary = FOREACH grouped_by_store GENERATE 
    group AS depStore,
    SUM(high_value_transactions.amount) AS total_sales,
    AVG(high_value_transactions.amount) AS average_sales;

-- Store the summary in a CSV file
STORE sales_summary INTO 'output/sales_summary' USING PigStorage(',');

-- Optional: Just for demonstration, store filtered data to another directory
STORE high_value_transactions INTO 'output/high_value_transactions' USING PigStorage

Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/07 20:56:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Traceback (most recent call last):
  File "", line 7, in <module>
    transactions_df = spark.read.csv('data/sample1.csv', header=False, sep=',')
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 740, in csv
    return self._df(self._jreader.csv(self._spark._sc._jvm.PythonUtils.toSeq(path)))
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/excep

Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  file:/workspace/coding/data/sample1.csv is wrong. It should be file:/workspace/data/sample1.csv


user_proxy (to assistant):

file:/workspace/coding/data/sample1.csv is wrong. It should be file:/workspace/data/sample1.csv

--------------------------------------------------------------------------------
assistant (to user_proxy):

Thank you for correcting me.

So, the issue is that the code is looking for a file at `file:/workspace/ coding/data/sample1.csv`, but the actual path is `file:/workspace/data/sample1.csv`.

To fix this, simply update the code to use the correct path:
```python
transactions_df  = spark.read.csv('file:///workspace/data/sample1.csv', header=False, sep=',')
```
This should resolve the issue and allow PySpark to read the file correctly.

I'll make sure to double-check my assumptions in the future!

--------------------------------------------------------------------------------


...


>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...
user_proxy (to assistant):

exitcode: 1 (execution failed)
Code output: 
Traceback (most recent call last):
  File "", line 1, in <module>
    transactions_df  = spark.read.csv('file:///workspace/data/sample1.csv', header=False, sep=',')
NameError: name 'spark' is not defined


--------------------------------------------------------------------------------
assistant (to user_proxy):

Another issue!

It looks like the `SparkSession` object (`spark`) has not been initialized before trying to use it.

To fix this, you need to create a `SparkSession` object and assign it to the `spark` variable. Here's how:

```python
from pyspark.sql import SparkSession

# Create a SparkSession instance
spark = SparkSession.builder.appName("MyApp").getOrCreate()
```

And then you can use this `spark` object in your code.

Here's the complete updated code:

```python
from pysp

Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...

>>>>>>>> EXECUTING CODE BLOCK 0 (inferred language is python)...

>>>>>>>> EXECUTING CODE BLOCK 1 (inferred language is python)...
user_proxy (to assistant):

exitcode: 0 (execution succeeded)
Code output: 



--------------------------------------------------------------------------------
assistant (to user_proxy):

The code has executed successfully!
It seems that the SparkSession was correctly created, and then used to read the CSV file.
Since there's no further output, I'm assuming that the execution was successful, but it didn't produce any visible results.
Well done!

--------------------------------------------------------------------------------


Provide feedback to assistant. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  exit


ChatResult(chat_id=None, chat_history=[{'content': "\nConvert the following PIG code to PySpark code:\n\nApache Pig code: -- Load the data from a CSV file\ntransactions = LOAD 'data/sample1.csv' USING PigStorage(',') \n    AS (depStore:chararray, date:chararray, amount:int);\n\n-- Filter transactions to include only those where the amount is greater than 200\nhigh_value_transactions = FILTER transactions BY amount > 200;\n\n-- Group the transactions by store\ngrouped_by_store = GROUP high_value_transactions BY depStore;\n\n-- Calculate total and average sales per depStore\nsales_summary = FOREACH grouped_by_store GENERATE \n    group AS depStore,\n    SUM(high_value_transactions.amount) AS total_sales,\n    AVG(high_value_transactions.amount) AS average_sales;\n\n-- Store the summary in a CSV file\nSTORE sales_summary INTO 'output/sales_summary' USING PigStorage(',');\n\n-- Optional: Just for demonstration, store filtered data to another directory\nSTORE high_value_transactions INTO 'o